In [16]:
import pandas as pd
import numpy as np
from dask import dataframe as dd
import time
import os
from dask.distributed import Client

In [ ]:
#This module provides an interface to the optional garbage collector. It provides the ability to disable the collector, tune the collection frequency, and set debugging options.

In [ ]:
# Just in case, to clean up RAM
import gc
gc.collect()

In [20]:
df.shape  #--> gives number of rows:cols

(9, 3)

In [21]:
df.head()

,Tran_Id,TranDate,TranAmnt
0,123,09-02-2022,200000
1,345,09-02-2022,300000
2,956,09-02-2022,500000
3,524,09-02-2022,800000
4,654,09-02-2022,200000


In [22]:
#SIZE OF DATA TO BE IMPORTED
print(os.path.getsize(r'C:\Users\Dell\Desktop\DaskPython\DaskTest\dask1.csv')/1024/1024/1024 , "GB")

2.1792948246002197e-07 GB


In [23]:
start = time.time()
df = pd.read_csv(r'C:\Users\Dell\Desktop\DaskPython\DaskTest\dask1.csv')
end = time.time()
print("Read csv without chunks: ",(end-start),"sec")

Read csv without chunks:  0.005986928939819336 sec


In [26]:
start = time.time()
chunk = pd.read_csv(r'C:\Users\Dell\Desktop\DaskPython\DaskTest\dask1.csv',chunksize=1000000)
end = time.time()
print("Read csv with chunks: ",(end-start),"sec")
pd_df = pd.concat(chunk)

Read csv with chunks:  0.002286672592163086 sec


In [27]:
type(chunk)

pandas.io.parsers.TextFileReader

In [ ]:
#The Client registers itself as the default Dask scheduler, and so runs all dask collections like dask.array, dask.bag, dask.dataframe and dask.delayed

In [29]:
client = Client(processes=False)
client

Client Scheduler: inproc://192.168.0.105/13348/1 Dashboard: http://192.168.0.105:8787/status,Cluster Workers: 1 Cores: 4 Memory: 7.90 GiB


In [37]:
start = time.time()
dask_df = dd.read_csv(r'C:\Users\Dell\Desktop\DaskPython\DaskTest\dask1.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")
pd_df = dask_df.compute()
print("Dask size:", dask_df.shape)

Read csv with dask:  0.01399683952331543 sec
Dask size: (Delayed('int-ec8ad790-5b96-4e1c-8201-6f2f055dd3be'), 3)


In [41]:
df1 = pd.read_csv(r'C:\Users\Dell\Desktop\DaskPython\DaskTest\dask1.csv')
print (df1)

   Tran_Id    TranDate  TranAmnt
0      123  09-02-2022    200000
1      345  09-02-2022    300000
2      956  09-02-2022    500000
3      524  09-02-2022    800000
4      654  09-02-2022    200000
5      123  09-02-2022    200000
6      345  09-02-2022    300000
7      654  09-02-2022    500000
8      123  15-02-2022    100000


In [42]:
df2 = pd.read_csv(r'C:\Users\Dell\Desktop\DaskPython\DaskTest\dask2.csv')
print(df2)

   Tran_Id    TranDate  TranAmnt
0      123  09-02-2022    200000
1      345  09-02-2022    260000
2      400  15-02-2022    500000
3      524  09-02-2022    800000
4      654  09-02-2022    200000
5      500  09-02-2022    260000
6      999  09-02-2022    300000
7      210  09-02-2022    700000
8      874  15-02-2022    100000


In [46]:
pd.merge(df1,df2 , on = 'Tran_Id', how = 'inner')

,Tran_Id,TranDate_x,TranAmnt_x,TranDate_y,TranAmnt_y
0,123,09-02-2022,200000,09-02-2022,200000
1,123,09-02-2022,200000,09-02-2022,200000
2,123,15-02-2022,100000,09-02-2022,200000
3,345,09-02-2022,300000,09-02-2022,260000
4,345,09-02-2022,300000,09-02-2022,260000
5,524,09-02-2022,800000,09-02-2022,800000
6,654,09-02-2022,200000,09-02-2022,200000
7,654,09-02-2022,500000,09-02-2022,200000


In [ ]:
#The isin() method checks if the Dataframe contains the specified value(s).

In [58]:
df1.apply(tuple,1).isin(df2.apply(tuple,1))

0     True
1    False
2    False
3     True
4     True
5     True
6    False
7    False
8    False
dtype: bool

In [57]:
start = time.time()
result = df1[df1.apply(tuple,1).isin(df2.apply(tuple,1))]
end = time.time()
print (result, '\n',(end-start),'sec')

   Tran_Id    TranDate  TranAmnt
0      123  09-02-2022    200000
3      524  09-02-2022    800000
4      654  09-02-2022    200000
5      123  09-02-2022    200000 
 0.006974220275878906 sec


In [61]:
start = time.time()
result2 = df1.merge(df2, how= 'inner', indicator = True)
end = time.time()
print (result2,'\n',(end-start),'sec')

   Tran_Id    TranDate  TranAmnt _merge
0      123  09-02-2022    200000   both
1      123  09-02-2022    200000   both
2      524  09-02-2022    800000   both
3      654  09-02-2022    200000   both 
 0.021940231323242188 sec


In [62]:
start = time.time()
result3 = df1.merge(df2, how= 'outer', indicator = True).loc[lambda v :v['_merge'] != 'both']
end = time.time()
print (result3,'\n',(end-start),'sec')

    Tran_Id    TranDate  TranAmnt      _merge
2       345  09-02-2022    300000   left_only
3       345  09-02-2022    300000   left_only
4       956  09-02-2022    500000   left_only
7       654  09-02-2022    500000   left_only
8       123  15-02-2022    100000   left_only
9       345  09-02-2022    260000  right_only
10      400  15-02-2022    500000  right_only
11      500  09-02-2022    260000  right_only
12      999  09-02-2022    300000  right_only
13      210  09-02-2022    700000  right_only
14      874  15-02-2022    100000  right_only 
 0.055845022201538086 sec


In [ ]:
#This returns all rows from both tables, join records from the left which have matching keys in the right table.When there is no Matching from any table NaN will be returned

In [87]:
start = time.time()
data_merge = df1.merge(df2, how = 'outer', left_on = "Tran_Id", right_on = "Tran_Id")
end = time.time()
print (data_merge,'\n',(end-start),'sec')


    Tran_Id  TranDate_x  TranAmnt_x  TranDate_y  TranAmnt_y
0       123  09-02-2022    200000.0  09-02-2022    200000.0
1       123  09-02-2022    200000.0  09-02-2022    200000.0
2       123  15-02-2022    100000.0  09-02-2022    200000.0
3       345  09-02-2022    300000.0  09-02-2022    260000.0
4       345  09-02-2022    300000.0  09-02-2022    260000.0
5       956  09-02-2022    500000.0         NaN         NaN
6       524  09-02-2022    800000.0  09-02-2022    800000.0
7       654  09-02-2022    200000.0  09-02-2022    200000.0
8       654  09-02-2022    500000.0  09-02-2022    200000.0
9       400         NaN         NaN  15-02-2022    500000.0
10      500         NaN         NaN  09-02-2022    260000.0
11      999         NaN         NaN  09-02-2022    300000.0
12      210         NaN         NaN  09-02-2022    700000.0
13      874         NaN         NaN  15-02-2022    100000.0 
 0.008977174758911133 sec


In [88]:
start = time.time()
data_merge2 = df1.merge(df2, how = 'inner', left_on = "TranAmnt", right_on = "TranAmnt")
end = time.time()
print (data_merge2,'\n',(end-start),'sec')


    Tran_Id_x  TranDate_x  TranAmnt  Tran_Id_y  TranDate_y
0         123  09-02-2022    200000        123  09-02-2022
1         123  09-02-2022    200000        654  09-02-2022
2         654  09-02-2022    200000        123  09-02-2022
3         654  09-02-2022    200000        654  09-02-2022
4         123  09-02-2022    200000        123  09-02-2022
5         123  09-02-2022    200000        654  09-02-2022
6         345  09-02-2022    300000        999  09-02-2022
7         345  09-02-2022    300000        999  09-02-2022
8         956  09-02-2022    500000        400  15-02-2022
9         654  09-02-2022    500000        400  15-02-2022
10        524  09-02-2022    800000        524  09-02-2022
11        123  15-02-2022    100000        874  15-02-2022 
 0.009974002838134766 sec


In [78]:
#compare files
with open(r'C:\Users\Dell\Desktop\DaskPython\DaskTest\dask1.csv') as t1, open(r'C:\Users\Dell\Desktop\DaskPython\DaskTest\dask2.csv') as t2:
    fileone = t1.readlines()
    filetwo = t2.readlines()

with open('update.csv','w') as outFile:
    for line in filetwo:
        if line not in fileone:
            outFile.write(line)
            end = time.time()

#update.csv will give difference of both files which is compared.

In [89]:
start = time.time()
result3 = pd.merge(df1,df2, on= ['Tran_Id', 'TranAmnt'], how = 'inner')
end = time.time()
print (result3,'\n',(end-start),'sec')

   Tran_Id  TranDate_x  TranAmnt  TranDate_y
0      123  09-02-2022    200000  09-02-2022
1      123  09-02-2022    200000  09-02-2022
2      524  09-02-2022    800000  09-02-2022
3      654  09-02-2022    200000  09-02-2022 
 0.00697779655456543 sec


In [92]:
start = time.time()
result5 = pd.merge(df1,df2, on= ['TranDate', 'TranAmnt', 'Tran_Id'], how = 'inner')
end = time.time()
print (result5,'\n',(end-start),'sec')

   Tran_Id    TranDate  TranAmnt
0      123  09-02-2022    200000
1      123  09-02-2022    200000
2      524  09-02-2022    800000
3      654  09-02-2022    200000 
 0.028254985809326172 sec
